In [17]:
# data processing tools
import string, os 
import pandas as pd
import numpy as np
np.random.seed(42)

# keras module for building LSTM 
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.utils as ku 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# surpress warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning) # Ignore warnings from libraries. 

import sys
sys.path.append("..")
import utils.requirement_functions as rf

In [18]:
# Loading data 
print("Loading data")
data_dir = os.path.join("..","data", "news_data")


Loading data


In [8]:
data_dir

'../data/news_data'

In [19]:
# Appending columns 
all_comments = []
for filename in os.listdir(data_dir):
    if 'Comments' in filename:
        comment_df = pd.read_csv(data_dir + "/" + filename) # joining data_dir / filename. ( Creating dataframe)
        all_comments.extend(list(comment_df["commentBody"].values)) # Creating a list of only comments. 

In [20]:
len(all_comments)

2176364

# Cleaning text

In [22]:
corpus = [rf.clean_text(x) for x in all_comments]
corpus[:10]

NameError: name 'string' is not defined